In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点 SDK：自定义训练图像分类模型用于批量预测并提供解释性

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_image_classification_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_image_classification_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_image_classification_batch_explain.ipynb">
      在 Google Cloud 笔记本中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK训练和部署自定义图像分类模型，以便进行批量预测并解释结果。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您将使用的数据集版本已经内置于TensorFlow中。训练好的模型会预测图像属于十种类别中的哪一种：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

### 目标

在本教程中，您将使用 Vertex SDK 在 Google 预构建的 Docker 容器中从 Python 脚本创建一个带有训练管道的自定义模型，然后对上传的模型进行批量预测并提供解释。您也可以选择使用 `gcloud` 命令行工具或在 Cloud Console 上在线创建自定义模型。

执行的步骤包括:

- 创建一个 Vertex 自定义作业以训练模型。
- 训练 TensorFlow 模型。
- 检索并加载模型工件。
- 查看模型评估。
- 设置解释参数。
- 将模型上传为 Vertex `Model` 资源。
- 进行带有解释的批量预测。

成本

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud 存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud 存储 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 来根据您的预期使用量生成成本估算。

### 设置您的本地开发环境

如果您正在使用Colab或Google Cloud笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容:

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行Jupyter笔记本

云存储指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简明的说明：

1. [安装和初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端窗口中的命令行中运行 `pip3 install jupyter`。

5. 要启动Jupyter，请在终端窗口中的命令行中运行 `jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

安装

安装最新版本的Python版Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的GA版本*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! apt-get update && apt-get install -y python3-opencv-headless
    ! apt-get install -y libgl1-mesa-dev
    ! pip3 install --upgrade opencv-python-headless $USER_FLAG

### 重新启动内核

一旦安装了额外的软件包，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在开始之前

GPU运行时

此教程不需要GPU运行时。

设置您的Google Cloud项目

**无论你使用什么笔记本环境，以下步骤都是必须的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个账户时，您可以获得$300的免费信用额度用于计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API、Compute Engine API和Cloud存储。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您正在本地运行此笔记本，您将需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 会将以 `!`为前缀的行作为shell命令运行，并解析以 `$`为前缀的Python变量。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于本笔记本其余部分的操作。以下是Vertex AI支持的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不会使用多区域存储桶来进行Vertex AI的训练。并非所有区域都支持所有Vertex AI服务。

了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您在进行实时教程会话，可能会使用一个共享的测试账户或项目。为了避免用户在创建的资源上发生命名冲突，您可以为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的 Google Cloud 帐户

**如果您正在使用 Google Cloud Notebooks**，您的环境已经经过身份验证。请跳过此步骤。

**如果您正在使用 Colab**，运行下面的单元格，并在提示时按照说明进行OAuth身份验证。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入一个名字，然后点击**创建**。

在**将此服务帐号授予对项目的访问权限**部分，点击角色下拉列表。在过滤框中键入“Vertex”，并选择**Vertex管理员**。在过滤框中键入“Storage Object Admin”，并选择**Storage Object Admin**。

点击创建。一个包含您的键的JSON文件将下载到本地环境中。

在下面的单元格中将您的服务帐号密钥的路径输入为 GOOGLE_APPLICATION_CREDENTIALS 变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储桶

**无论您使用什么笔记本环境，都需要以下步骤。**

当您初始化用于Python的Vertex SDK时，需要指定一个云存储暂存桶。暂存桶是在会话之间保留与您的数据集和模型资源相关的所有数据的地方。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查Cloud Storage存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。

### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

## 初始化Python的Vertex SDK

为您的项目和相应的存储桶初始化Python的Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机实例的GPU数量。例如，要使用一个含有4个Nvidia Telsa K80 GPU的容器映像分配给每个VM，您应该指定：

(aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`以使用一个在CPU上运行的容器映像。

了解更多关于您所在地区的[硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)

*注意*: 在2.3版本之前的TF版本中，对于GPU支持加载自定义模型将导致本教程失败。这是一个已知问题，在TF 2.3版本中已修复——这是由生成在serving函数中的静态图操作引起的。如果您在自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预先构建的容器

设置用于训练和预测的预先构建的 Docker 容器映像。

获取最新列表，请参阅 [用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

获取最新列表，请参阅 [用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的虚拟机的计算资源。
  - `机器类型`
     - `n1-standard`：每个 vCPU 3.75GB 内存。
     - `n1-highmem`：每个 vCPU 6.5GB 内存。
     - `n1-highcpu`：每个 vCPU 0.9GB 内存。
  - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：以下机器类型不支持用于训练：*
  - `standard`：2个 vCPUs
  - `highcpu`：2、4 和 8 个 vCPUs

*注意：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPUs。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您已经准备好开始创建自己的自定义模型并为CIFAR10进行训练。

### 检查培训软件包

#### 包布局

在开始培训之前，您将查看如何为自定义培训任务组装Python软件包。解压后，软件包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将软件包安装到Docker镜像的操作环境中的说明。

文件 `trainer/task.py` 是执行自定义培训任务的Python脚本。 *注意*，当在工作池规范中引用它时，我们将目录斜杠替换为点(`trainer.task`)，并删除文件后缀（`.py`）。

#### 包装配

在下面的单元格中，您将组装培训软件包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 的内容

在下一个单元格中，您将编写训练脚本 task.py 的内容。我们不会详细解释，只是让您浏览。总结如下：

- 从命令行中获取保存模型 artifacts 的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF Datasets（tfds）中加载 CIFAR10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分发策略。
- 使用参数 `args.epochs` 和 `args.steps` 训练模型（`fit()`）。
- 将训练好的模型保存至指定模型目录（`save(args.model_dir)`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的tar文件，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar10.tar.gz

### 创建和运行自定义训练作业

为了训练一个自定义模型，您需要执行两个步骤：1) 创建一个自定义训练作业，2) 运行该作业。

#### 创建自定义训练作业

使用`CustomTrainingJob`类创建一个自定义训练作业，需要以下参数：

- `display_name`：自定义训练作业的可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：训练容器镜像的软件包需求（例如，pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aip.CustomTrainingJob(
    display_name="cifar10_" + TIMESTAMP,
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

### 准备你的命令行参数

现在为你的自定义训练容器定义命令行参数：

- `args`：传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：对于我们的演示，我们使用这个命令行参数来指定存储模型文件的位置。
      - 直接方式：将云存储位置作为命令行参数传递给你的训练脚本（设置变量 `DIRECT = True`），或
      - 间接方式：服务将云存储位置作为环境变量 `AIP_MODEL_DIR` 传递给你的训练脚本（设置变量 `DIRECT = False`）。在这种情况下，你在作业规范中告诉服务模型文件位置。
  - `"--epochs=" + EPOCHS`：训练时的周期数。
  - `"--steps=" + STEPS`：每个周期的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_NAME, TIMESTAMP)

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

#### 运行自定义训练作业

接下来，通过调用`run`方法，并传入以下参数来运行自定义作业以启动训练作业：

- `args`：要传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作人员副本的加速器数量。
- `base_output_dir`：写入模型工件的 Cloud 存储位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶中加载它，然后您可以执行一些操作，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，将保存模型的 Cloud Storage 路径（由 `MODEL_DIR` 指定）传递给它。

In [ ]:
import tensorflow as tf

local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在找出模型的表现如何。

### 加载评估数据

您将使用 `load_data()` 方法从 `tf.keras.datasets` 加载 CIFAR10 测试数据。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：图像数据和相应的标签。

您不需要训练数据，这就是为什么我们把它加载为 `(_, _)`。

在您可以对数据进行评估之前，您需要对其进行预处理：

`x_test`:
1. 通过将每个像素除以 255 对像素数据进行标准化（重新缩放）。这将用介于 0 和 1 之间的 32 位浮点数替换每个单字节整数像素。

`y_test`:<br/>
2. 标签当前是标量（稀疏）。如果您回头看 `trainer/task.py` 脚本中的 `compile()` 步骤，您会发现它是为稀疏标签编译的。所以我们不需要再做任何处理。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10

(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

### 执行模型评估

现在评估定制作业中的模型表现如何。

In [ ]:
local_model.evaluate(x_test, y_test)

### 图像数据的服务功能

要将图像传递给预测服务，您需要将压缩（例如，JPEG）图像字节编码为base 64 — 这样可以使内容在通过网络传输二进制数据时免受修改。由于这个部署的模型期望输入数据为原始（未压缩）字节，您需要确保将base 64编码的数据在传递给部署模型的输入之前转换回原始字节。

为了解决这个问题，您需要定义一个服务函数（`serving_fn`）并将它附加到模型作为预处理步骤。添加一个 `@tf.function` 装饰器，以便服务函数与底层模型融合（而不是在CPU上游）。

当您发送预测或解释请求时，请求的内容被base 64解码为一个Tensorflow字符串（`tf.string`），然后传递给服务函数（`serving_fn`）。服务函数将 `tf.string` 预处理为原始（未压缩）的numpy字节（`preprocess_fn`）以匹配模型的输入要求：
- `io.decode_jpeg` - 解压JPG图像，返回一个具有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值更改为float 32。
- `image.resize` - 将图像调整大小以匹配模型的输入形状。
- `resized / 255.0` - 将像素数据重新缩放（归一化）在0和1之间。

在这一点上，数据可以传递给模型（`m_call`）。

#### XAI 签名

当服务函数与底层模型一起保存（`tf.saved_model.save`）时，您需指定服务函数的输入层作为签名`serving_default`。

对于XAI图像模型，您需要保存从服务函数中保存两个额外签名：

- `xai_preprocess`：服务函数中的预处理函数。
- `xai_model`：调用模型的具体函数。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    rescale = tf.cast(resized / 255.0, tf.float32)
    return rescale


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(local_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    local_model,
    model_path_to_deploy,
    signatures={
        "serving_default": serving_fn,
        # Required for XAI
        "xai_preprocess": preprocess_fn,
        "xai_model": m_call,
    },
)

获取服务功能的签名

您可以通过重新加载模型到内存并查询相应的签名来获取模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务函数而不是模型，因此您需要知道服务功能的输入层名称 - 在稍后进行预测请求时将使用。

您还需要知道服务功能的输入和输出层的名称，以构建解释元数据 - 这将在随后讨论。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

input_name = local_model.input.name
print("Model input name:", input_name)
output_name = local_model.output.name
print("Model output name:", output_name)

### 说明规范

要在进行预测时获得解释，您必须启用解释功能，并在将自定义模型上传到 Vertex `Model` 资源时设置相应的设置。这些设置被称为解释元数据，包括：

- `parameters`：这是用于解释您的模型的可解释性算法的规范。您可以在以下选项中选择：
  - Shapley - *注意*，不推荐用于图像数据 -- 可能运行时间很长
  - XRAI
  - Integrated Gradients
- `metadata`：这是用于应用算法在您的自定义模型上的规范。

#### 解释参数

让我们首先深入了解可解释性算法的设置。

#### Shapley

将结果的 credit 分配给每个特征，并考虑不同特征的排列组合。这种方法提供了对确切 Shapley 值的采样近似。

使用情况：
  - 对表格数据进行分类和回归。

参数：

- `path_count`：这是算法处理的特征路径数。要获得精确的 Shapley 值近似，需要 M! 条路径，其中 M 是特征数量。对于 CIFAR10 数据集，这将是 784 (28*28)。

对于任何非平凡数量的特征，这个计算开销太大。您可以将特征路径数减少到 M * `path_count`。

#### Integrated Gradients

一种基于梯度的方法，可以有效计算具有与 Shapley 值相同公理属性的特征归因。

使用情况：
  - 对表格数据进行分类和回归。
  - 对图像数据进行分类。

参数：

- `step_count`：这是用于近似剩余和的步数。步数越多，积分近似越准确。一般的经验法则是 50 步，但随着步数的增加，计算时间也会增加。

#### XRAI

基于集成梯度方法，XRAI 评估图像的重叠区域以创建一个显著性地图，突出显示图像的相关区域而不是像素。

使用情况：

  - 对图像数据进行分类。

参数：

- `step_count`：这是用于近似剩余和的步数。步数越多，积分近似越准确。一般的经验法则是 50 步，但随着步数的增加，计算时间也会增加。

在下一个代码单元格中，将变量 `XAI` 设置为您将在自定义模型上使用的解释性算法。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aip.explain.ExplanationParameters(PARAMETERS)

#### 解释元数据

让我们首先深入了解解释元数据，它包括：

- `outputs`：在输出中的一个标量值，用于归因 -- 要解释什么。例如，在分类的概率输出\[0.1, 0.2, 0.7\]中，我们想要解释0.7。考虑以下公式，其中输出是`y`，这就是我们要解释的东西。

    y = f(x)

考虑以下公式，其中输出是`y`和`z`。由于我们只能对一个标量值进行归因，我们必须选择要解释输出`y`还是`z`。假设在这个例子中模型是目标检测，y和z分别是边界框和对象分类。您需要选择要解释的这两个输出中的哪一个。

    y, z = f(x)

`outputs`的字典格式是：

    { "outputs": { "[your_display_name]":
                   "output_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]：您为要解释的输出指定的人类可读名称。一个常见的例子是“概率”。
 -  “output_tensor_name”：用于识别要解释的输出层的键/值字段。
 -  [layer]：要解释的输出层。在单一任务模型中，比如表格回归器，它是模型中的最后（最顶层）层。
</blockquote>

- `inputs`：用于归因的特征 -- 它们如何影响输出。考虑以下公式，其中`a`和`b`是特征。我们必须选择要解释如何贡献的特征。假设这个模型被部署用于A/B测试，在此情况下，`a`是预测的数据项，`b`标识模型实例是A还是B。您会选择`a`（或其部分）作为特征，而不是`b`，因为它不会对预测有贡献。

    y = f(a,b)

`inputs`的最小字典格式是：

    { "inputs": { "[your_display_name]":
                  "input_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]：您为要解释的输入指定的人类可读名称。一个常见的例子是“特征”。
 -  “input_tensor_name”：用于识别用于特征归因的输入层的键/值字段。
 -  [layer]：用于特征归因的输入层。在单一输入张量模型中，这是模型中的第一（最底层）层。
</blockquote>

由于模型的输入是图像，您可以指定以下附加字段作为报告/可视化辅助工具：

<blockquote>
 - "modality": "image": 表示字段值是图像数据。
</blockquote>

In [ ]:
random_baseline = np.random.rand(32, 32, 3)
input_baselines = [{"number_vaue": x} for x in random_baseline]

INPUT_METADATA = {"input_tensor_name": CONCRETE_INPUT, "modality": "image"}

OUTPUT_METADATA = {"output_tensor_name": serving_output}

input_metadata = aip.explain.ExplanationMetadata.InputMetadata(INPUT_METADATA)
output_metadata = aip.explain.ExplanationMetadata.OutputMetadata(OUTPUT_METADATA)

metadata = aip.explain.ExplanationMetadata(
    inputs={"image": input_metadata}, outputs={"class": output_metadata}
)

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到`Model`资源，传入以下参数：

- `display_name`: `Model`资源的可读名称。
- `artifact`: 训练模型工件的云存储位置。
- `serving_container_image_uri`: 提供服务的容器映像。
- `sync`: 是否异步或同步执行上传。
- `explanation_parameters`: 为`Model`的预测配置解释的参数。
- `explanation_metadata`: 描述用于解释的`Model`的输入和输出的元数据。

如果`upload()`方法是异步运行的，您可以随后使用`wait()`方法阻塞，直到完成为止。

In [ ]:
model = aip.Model.upload(
    display_name="cifar10_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    explanation_parameters=parameters,
    explanation_metadata=metadata,
    sync=False,
)

model.wait()

获取测试项

您将从数据集的测试（留存）部分中选取示例作为测试项。

In [ ]:
test_image_1 = x_test[0]
test_label_1 = y_test[0]
test_image_2 = x_test[1]
test_label_2 = y_test[1]
print(test_image_1.shape)

### 准备请求内容
你要发送 CIFAR10 图像作为压缩的 JPG 图像，而不是原始的未压缩字节：

- `cv2.imwrite`：使用 openCV 将未压缩的图像写入磁盘，作为压缩的 JPEG 图像。
 - 从\[0,1)范围中将图像数据反归一化为 [0,255)。
 - 将32位浮点数值转换为8位无符号整数。

In [ ]:
import cv2

cv2.imwrite("tmp1.jpg", (test_image_1 * 255).astype(np.uint8))
cv2.imwrite("tmp2.jpg", (test_image_2 * 255).astype(np.uint8))

### 复制测试项目

对于批量预测，请将测试项目复制到您的云存储桶中。

In [ ]:
! gsutil cp tmp1.jpg $BUCKET_NAME/tmp1.jpg
! gsutil cp tmp2.jpg $BUCKET_NAME/tmp2.jpg

test_item_1 = BUCKET_NAME + "/tmp1.jpg"
test_item_2 = BUCKET_NAME + "/tmp2.jpg"

### 制作批量输入文件

现在制作一个批量输入文件，您将把它存储在您的本地云存储桶中。批量输入文件只能是以JSONL格式。对于JSONL文件，您为每个数据项（实例）在每一行中创建一个字典条目。字典包含键/值对：

- `input_name`：底层模型输入层的名称。
- `'b64'`：指示内容为base64编码的键。
- `content`：将压缩的JPG图像字节作为base64编码的字符串。

预测请求中的每个实例都是以下形式的字典条目：

{服务输入层名称: {'b64'：内容}}

为了将图像数据传递给预测服务，您需要将字节编码为base64——这样在通过网络传输二进制数据时，内容就不会被修改。

- `tf.io.read_file`：将压缩的JPG图像以原始字节的方式读入内存。
- `base64.b64encode`：将原始字节编码为base64编码的字符串。

In [ ]:
import base64
import json

gcs_input_uri = BUCKET_NAME + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    bytes = tf.io.read_file(test_item_1)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    data = {serving_input: {"b64": b64str}}
    f.write(json.dumps(data) + "\n")
    bytes = tf.io.read_file(test_item_2)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    data = {serving_input: {"b64": b64str}}
    f.write(json.dumps(data) + "\n")

### 发送批量预测请求

现在您的模型资源已经训练好，您可以通过调用batch_predict()方法来进行批量预测，需要以下参数：

- `job_display_name`: 批量预测作业的可读名称。
- `gcs_source`: 一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`: 用于存储批量预测结果的Cloud Storage位置。
- `instances_format`: 输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`: 输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `machine_type`: 用于训练的机器类型。
- `sync`: 如果设置为True，调用将会在等待异步批量作业完成时阻塞。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name="cifar10_" + TIMESTAMP,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_NAME,
    instances_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    generate_explanation=True,
    sync=False,
)

print(batch_predict_job)

### 等待批量预测作业完成

接下来，等待批量作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

获取解释

接下来，从完成的批量预测作业中获取解释结果。

结果被写入了您在批量预测请求中指定的云存储输出桶。您可以调用iter_outputs()方法来获取生成结果的每个云存储文件的列表。每个文件以CSV格式包含一个或多个解释请求：

- CSV表头 + 预测标签
- CSV行 + 解释，每个预测请求一个

In [ ]:
import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

explanation_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("explanation"):
        explanation_results.append(blob.name)

tags = list()
for explanation_result in explanation_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{explanation_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            print(line)

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- AutoML训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME